In [5]:
import cv2
import numpy as np

def bilateral_filtering(
    img: np.uint8,
    spatial_variance: float,
    intensity_variance: float,
    kernel_size: int,
) -> np.uint8:
    img = img / 255.0
    img_filtered = np.zeros(img.shape, dtype=np.float32)

    half_kernel_size = kernel_size // 2

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            pixel_value = img[i, j]
            weighted_sum = 0.0
            total_weight = 0.0

            for x in range(-half_kernel_size, half_kernel_size + 1):
                for y in range(-half_kernel_size, half_kernel_size + 1):
                    ni = i + x
                    nj = j + y

                    if ni >= 0 and ni < img.shape[0] and nj >= 0 and nj < img.shape[1]:
                        neighbor_value = img[ni, nj]
                        spatial_distance = np.sqrt(x**2 + y**2)
                        intensity_distance = np.abs(pixel_value - neighbor_value)

                        spatial_weight = np.exp(-(spatial_distance**2) / (2 * spatial_variance))
                        intensity_weight = np.exp(-(intensity_distance**2) / (2 * intensity_variance))

                        weight = spatial_weight * intensity_weight
                        weighted_sum += neighbor_value * weight
                        total_weight += weight

            img_filtered[i, j] = weighted_sum / total_weight

    img_filtered = np.uint8(img_filtered * 255)
    return img_filtered

if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA) # reduce image size for saving computation time
    cv2.imwrite('results/im_original.png', img) # save image 

    # Generate Gaussian noise
    noise = np.random.normal(0, 0.6, img.shape).astype('uint8')

    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)

    # Bilateral filtering
    spatial_variance = 30.0 # sigma_s^2
    intensity_variance = 0.5 # sigma_r^2
    kernel_size = 7
    img_bi = bilateral_filtering(img_noise, spatial_variance, intensity_variance, kernel_size)
    cv2.imwrite('results/im_bilateral357.png', img_bi)


In [1]:
"""
CS 4391 Homework 2 Programming: Part 3 - bilateral filter
Implement the bilateral_filtering() function in this python script
"""
 
import cv2
import numpy as np
import math

def bilateral_filtering(
    img: np.uint8,
    spatial_variance: float,
    intensity_variance: float,
    kernel_size: int,
) -> np.uint8:
    """
    Homework 2 Part 3
    Compute the bilaterally filtered image given an input image, kernel size, spatial variance, and intensity range variance
    """

    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape) # Placeholder of the filtered image
    
    
    
    
    # Todo: For each pixel position [i, j], you need to compute the filtered output: img_filtered[i, j]
    
    # step 1: compute kernel_size x kernel_size spatial and intensity range weights of the bilateral filter in 
    # terms of spatial_variance and intensity_variance. 
    
    # step 2: compute the filtered pixel img_filtered[i, j] using the obtained kernel weights and the neighboring
    # pixels of img[i, j] in the kernel_size x kernel_size local window
    
    # The bilateral filtering formula can be found in slide 15 of lecture 6
    # Tip: use zero-padding to address the black border issue.

    # ********************************
    # Your code is here.
    # ********************************
    
    # Iterate through each pixel in the image
    height, width = img.shape

    for i in range(height):
        for j in range(width):
            pixel_value = 0.0
            total_weight = 0.0
            
            # Define the region of the kernel around the current pixel
            i_min = max(i - kernel_size // 2, 0)
            i_max = min(i + kernel_size // 2, height - 1)
            j_min = max(j - kernel_size // 2, 0)
            j_max = min(j + kernel_size // 2, width - 1)
            
            for x in range(i_min, i_max + 1):
                for y in range(j_min, j_max + 1):
                    spatial_dist = math.sqrt((i - x) ** 2 + (j - y) ** 2)
                    intensity_dist = abs(img[i, j] - img[x, y])
                    
                    # Compute spatial and intensity weights
                    spatial_weight = math.exp(-0.5 * (spatial_dist**2) / (spatial_variance**2))
                    intensity_weight = math.exp(-0.5 * (intensity_dist**2) / (intensity_variance**2))
                    
                    # Combine spatial and intensity weights
                    weight = spatial_weight * intensity_weight
                    pixel_value += img[x, y] * weight
                    total_weight += weight
            
            img_filtered[i, j] = pixel_value / total_weight
    
    
    
    img_filtered = img_filtered * 255
    img_filtered = np.uint8(img_filtered)
    return img_filtered

 
if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA) # reduce image size for saving your computation time
    cv2.imwrite('results/im_original.png', img) # save image 
    
    # Generate Gaussian noise
    noise = np.random.normal(0,0.6,img.size)
    noise = noise.reshape(img.shape[0],img.shape[1]).astype('uint8')
   
    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)
    
    # Bilateral filtering
    spatial_variance = 30 # signma_s^2
    intensity_variance = 0.5 # sigma_r^2
    kernel_size = 7
    img_bi = bilateral_filtering(img_noise, spatial_variance, intensity_variance, kernel_size)
    cv2.imwrite('results/im_bilateral88.png', img_bi)

In [2]:
import cv2
import numpy as np

def bilateral_filtering(
    img: np.uint8,
    spatial_variance: float,
    intensity_variance: float,
    kernel_size: int,
) -> np.uint8:
    img = img / 255.0  # Normalize image to range [0, 1]
    img_filtered = np.zeros_like(img)  # Placeholder of the filtered image

    # Calculate half-size of the kernel (used for indexing)
    kernel_half_size = kernel_size // 2

    # Pad the input image with zeros to handle border pixels
    img_padded = np.pad(img, ((kernel_half_size, kernel_half_size), (kernel_half_size, kernel_half_size)), mode='constant')

    height, width = img.shape

    # Iterate over each pixel in the input image
    for i in range(height):
        for j in range(width):
            pixel_intensity = img_padded[i + kernel_half_size, j + kernel_half_size]  # Intensity of the current pixel
            weighted_sum = 0.0
            sum_of_weights = 0.0

            # Iterate over the kernel window
            for m in range(-kernel_half_size, kernel_half_size + 1):
                for n in range(-kernel_half_size, kernel_half_size + 1):
                    # Calculate spatial weight (Gaussian)
                    spatial_weight = np.exp(-((m ** 2 + n ** 2) / (2 * spatial_variance ** 2)))

                    # Calculate intensity weight (Gaussian)
                    intensity_weight = np.exp(-((img_padded[i + m + kernel_half_size, j + n + kernel_half_size] - pixel_intensity) ** 2) / (2 * intensity_variance ** 2))

                    # Calculate the bilateral filter weight
                    bilateral_weight = spatial_weight * intensity_weight

                    # Accumulate the weighted sum
                    weighted_sum += img_padded[i + m + kernel_half_size, j + n + kernel_half_size] * bilateral_weight
                    sum_of_weights += bilateral_weight

            # Calculate the filtered pixel value
            img_filtered[i, j] = weighted_sum / sum_of_weights

    img_filtered = np.uint8(img_filtered * 255)  # Denormalize and convert to uint8
    return img_filtered

if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0)  # Read grayscale image
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)  # Resize image

    cv2.imwrite('results/im_original.png', img)  # Save original image

    # Generate Gaussian noise
    noise = np.random.normal(0, 0.6, img.shape).astype('uint8')

    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)

    cv2.imwrite('results/im_noisy.png', img_noise)  # Save noisy image

    # Bilateral filtering
    spatial_variance = 30.0  # sigma_s^2
    intensity_variance = 0.5  # sigma_r^2
    kernel_size = 7

    img_bi = bilateral_filtering(img_noise, spatial_variance, intensity_variance, kernel_size)
    cv2.imwrite('results/im_bilateral77.png', img_bi)  # Save filtered image


In [15]:
"""
CS 4391 Homework 2 Programming: Part 3 - bilateral filter
Implement the bilateral_filtering() function in this python script
"""
 
import cv2
import numpy as np
import math

def bilateral_filtering(
    img: np.uint8,
    spatial_variance: float,
    intensity_variance: float,
    kernel_size: int,
) -> np.uint8:
    
    """
    Homework 2 Part 3
    Compute the bilaterally filtered image given an input image, kernel size, spatial variance, and intensity range variance
    """

    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape) # Placeholder of the filtered image
    
    # Todo: For each pixel position [i, j], you need to compute the filtered output: img_filtered[i, j]
    # step 1: compute kernel_sizexkernel_size spatial and intensity range weights of the bilateral filter 
        #in terms of spatial_variance and intensity_variance. 
    # step 2: compute the filtered pixel img_filtered[i, j] using the obtained kernel weights and the neighboring 
        #pixels of img[i, j] in the kernel_sizexkernel_size local window
    # The bilateral filtering formula can be found in slide 15 of lecture 6
    # Tip: use zero-padding to address the black border issue.
    # ********************************
    # Your code is here.
    # ********************************
      # Calculate half-size of the kernel (used for indexing)
    kernel_half_size = (kernel_size // 2)

    # Pad the input image with zeros to handle border pixels
    pad = np.pad(img, (((kernel_size // 2), (kernel_size // 2)), ((kernel_size // 2), (kernel_size // 2))), mode='constant')
    h, w = img.shape
    for i in range(h):
        for j in range(w):
            sum2 = 0
            kernel_weight = 0

            for x in range(-(kernel_size // 2), (kernel_size // 2)+ 1):
                for y in range(-(kernel_size // 2), (kernel_size // 2)+ 1):
                    if (i + x) >= 0 and (i + x) < h and (j + y) >= 0 and (j + y) < w:   
                        spatial_weight = np.exp(-((np.sqrt(x**2 + y**2))**2) / (2 * spatial_variance))
                        intensity_weight = np.exp(-((np.abs(img[i, j] - img[(i + x), (j + y)]))**2) / (2 * intensity_variance))
                        sum2 += img[(i + x), (j + y)] * (spatial_weight * intensity_weight)
                        kernel_weight += (spatial_weight * intensity_weight)

            img_filtered[i, j] = sum2 / kernel_weight

    img_filtered = img_filtered * 255
    img_filtered = np.uint8(img_filtered)
    return img_filtered

if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA) # reduce image size for saving your computation time
    cv2.imwrite('results/im_original.png', img) # save image 
    
    # Generate Gaussian noise
    noise = np.random.normal(0,0.6,img.size)
    noise = noise.reshape(img.shape[0],img.shape[1]).astype('uint8')
   
    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)
    
    # Bilateral filtering
    spatial_variance = 30 # signma_s^2
    intensity_variance = 0.5 # sigma_r^2
    kernel_size = 7
    img_bi = bilateral_filtering(img_noise, spatial_variance, intensity_variance, kernel_size)
    cv2.imwrite('results/im_bilateralendpad.png', img_bi)

In [3]:
import cv2
import numpy as np
import math

def bilateral_filtering(
    img: np.uint8,
    spatial_variance: float,
    intensity_variance: float,
    kernel_size: int,
) -> np.uint8:
    img = img / 255.0  # Normalize image to [0, 1]
    img_filtered = np.zeros_like(img)  # Placeholder for the filtered image
    height, width = img.shape
    pad = kernel_size // 2

    for y in range(height):
        for x in range(width):
            pixel = img[y, x]
            weighted_sum = 0.0
            sum_of_weights = 0.0

            for j in range(-pad, pad + 1):
                for i in range(-pad, pad + 1):
                    neighbor_x = x + i
                    neighbor_y = y + j

                    if 0 <= neighbor_x < width and 0 <= neighbor_y < height:
                        neighbor_pixel = img[neighbor_y, neighbor_x]

                        # Calculate spatial and intensity weights
                        spatial_weight = math.exp(-((i ** 2 + j ** 2) / (2 * spatial_variance)))
                        intensity_weight = math.exp(-((pixel - neighbor_pixel) ** 2) / (2 * intensity_variance))

                        # Compute the bilateral filter value
                        bilateral_value = spatial_weight * intensity_weight * neighbor_pixel

                        weighted_sum += bilateral_value
                        sum_of_weights += spatial_weight * intensity_weight

            img_filtered[y, x] = weighted_sum / sum_of_weights

    img_filtered = (img_filtered * 255.0).astype(np.uint8)  # Rescale to [0, 255]
    return img_filtered

if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0)  # Read gray image
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)  # Reduce image size for saving computation time
    cv2.imwrite('results/im_original.png', img)  # Save original image

    # Generate Gaussian noise
    noise = np.random.normal(0, 0.6, img.shape).astype('float32')
    
    # Convert img to float32 and add the generated Gaussian noise
    img = img.astype('float32')
    img_noise = cv2.add(img, noise).astype(np.uint8)
    cv2.imwrite('results/im_noisy.png', img_noise)

    # Bilateral filtering
    spatial_variance = 30.0  # sigma_s^2
    intensity_variance = 0.5  # sigma_r^2
    kernel_size = 7
    img_bi = bilateral_filtering(img_noise, spatial_variance, intensity_variance, kernel_size)
    cv2.imwrite('results/im_bilateral567.png', img_bi)


In [7]:
def bilateral_filtering(
    img: np.uint8,
    spatial_variance: float,
    intensity_variance: float,
    kernel_size: int,
) -> np.uint8:
    """
    Homework 2 Part 3
    Compute the bilaterally filtered image given an input image, kernel size, spatial variance, and intensity range variance
    """

    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape)  # Placeholder of the filtered image

    # Define the padding size based on the kernel size
    padding = kernel_size // 2

    # Add zero-padding to the input image
    img_padded = np.pad(img, [(padding, padding), (padding, padding)], mode="constant")

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            sum2 = 0
            kernel_weight = 0

            for x in range(-padding, padding + 1):
                for y in range(-padding, padding + 1):
                    # Calculate the corresponding position in the padded image
                    x_padded = i + x + padding
                    y_padded = j + y + padding

                    spatial_weight = np.exp(-((np.sqrt(x**2 + y**2))**2) / (2 * spatial_variance))
                    intensity_weight = np.exp(-((np.abs(img[i, j] - img_padded[x_padded, y_padded]))**2) / (2 * intensity_variance))
                    sum2 += img_padded[x_padded, y_padded] * (spatial_weight * intensity_weight)
                    kernel_weight += (spatial_weight * intensity_weight)

            img_filtered[i, j] = sum2 / kernel_weight
    img_filtered = img_filtered[padding:-padding, padding:-padding]

    img_filtered = img_filtered * 255
    img_filtered = np.uint8(img_filtered)
    return img_filtered

if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0)  # Read gray image
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)  # Reduce image size for saving computation time
    cv2.imwrite('results/im_original.png', img)  # Save original image

    # Generate Gaussian noise
    noise = np.random.normal(0, 0.6, img.shape).astype('float32')
    
    # Convert img to float32 and add the generated Gaussian noise
    img = img.astype('float32')
    img_noise = cv2.add(img, noise).astype(np.uint8)
    cv2.imwrite('results/im_noisy.png', img_noise)

    # Bilateral filtering
    spatial_variance = 30.0  # sigma_s^2
    intensity_variance = 0.5  # sigma_r^2
    kernel_size = 7
    img_bi = bilateral_filtering(img_noise, spatial_variance, intensity_variance, kernel_size)
    cv2.imwrite('results/im_bilateralpad.png', img_bi)


In [ ]:
import cv2
import numpy as np
import math

def nlm_filtering(
    img: np.uint8,
    intensity_variance: float,
    patch_size: int,
    window_size: int,
) -> np.uint8:
    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape) # Placeholder of the filtered image

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            patch = img[max(0, i - patch_size):min(img.shape[0], i + patch_size + 1),
                        max(0, j - patch_size):min(img.shape[1], j + patch_size + 1)]

            weighted_sum = 0
            normalization_factor = 0

            for m in range(max(0, i - window_size), min(img.shape[0], i + window_size + 1)):
                for n in range(max(0, j - window_size), min(img.shape[1], j + window_size + 1)):
                    neighbor_patch = img[max(0, m - patch_size):min(img.shape[0], m + patch_size + 1),
                                         max(0, n - patch_size):min(img.shape[1], n + patch_size + 1)]

                    if patch.shape == neighbor_patch.shape:
                        diff = patch - neighbor_patch
                        diff_squared = np.sum(diff ** 2)

                        weight = math.exp(-diff_squared / (2 * intensity_variance ** 2))
                        weighted_sum += weight * img[m, n]
                        normalization_factor += weight

            img_filtered[i, j] = weighted_sum / normalization_factor

    img_filtered = np.uint8(img_filtered * 255)
    return img_filtered

if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA) # reduce image size for saving your computation time
    cv2.imwrite('results/im_original.png', img) # save image 
    
    # Generate Gaussian noise
    noise = np.random.normal(0,0.6,img.size)
    noise = noise.reshape(img.shape[0],img.shape[1]).astype('uint8')
   
    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)
    
    # NLM filtering
    intensity_variance = 1
    patch_size = 5 # small image patch size
    window_size = 15 # search window size
    img_nlm = nlm_filtering(img_noise, intensity_variance, patch_size, window_size)
    cv2.imwrite('results/im_nlm3.png', img_nlm)


In [ ]:
def nlm_filtering(
    img: np.uint8,
    intensity_variance: float,
    patch_size: int,
    window_size: int,
) -> np.uint8:
    """
    Homework 2 Part 4
    Compute the filtered image given an input image, kernel size of image patch, spatial variance, and intensity range variance
    """

    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape) # Placeholder of the filtered image
    
    # Loop through each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            pixel_value = img[i, j]
            weighted_sum = 0.0
            normalization_factor = 0.0
            
            # Define the search window
            i_min = max(0, i - window_size // 2)
            i_max = min(img.shape[0], i + window_size // 2 + 1)
            j_min = max(0, j - window_size // 2)
            j_max = min(img.shape[1], j + window_size // 2 + 1)
            
            # Loop through the pixels in the search window
            for x in range(i_min, i_max):
                for y in range(j_min, j_max):
                    patch = img[x:x+patch_size, y:y+patch_size]  # Corrected patch definition
                    
                    # Compute the Gaussian weight based on intensity_variance
                    intensity_difference = patch - pixel_value
                    weight = np.exp(-np.sum(intensity_difference ** 2) / (2 * intensity_variance ** 2))
                    
                    # Accumulate the weighted pixel values and weights
                    weighted_sum += weight * patch[0, 0]  # You can choose any pixel in the patch
                    normalization_factor += weight
            
            # Normalize and assign the filtered value
            img_filtered[i, j] = weighted_sum / normalization_factor
    
    img_filtered = img_filtered * 255
    img_filtered = np.uint8(img_filtered)
    return img_filtered

if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA) # reduce image size for saving your computation time
    cv2.imwrite('results/im_original.png', img) # save image 
    
    # Generate Gaussian noise
    noise = np.random.normal(0,0.6,img.size)
    noise = noise.reshape(img.shape[0],img.shape[1]).astype('uint8')
   
    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)
    
    # Bilateral filtering
    intensity_variance = 1
    patch_size = 5 # small image patch size
    window_size = 15 # serach window size
    img_bi = nlm_filtering(img_noise, intensity_variance, patch_size, window_size)
    cv2.imwrite('results/im_nlmlast.png', img_bi)

In [7]:
"""
CS 4391 Homework 2 Programming: Part 4 - non-local means filter
Implement the nlm_filtering() function in this python script
"""
 
import cv2
import numpy as np
import math

def nlm_filtering(
    img: np.uint8,
    intensity_variance: float,
    patch_size: int,
    window_size: int,
) -> np.uint8:
    """
    Homework 2 Part 4
    Compute the filtered image given an input image, kernel size of image patch, spatial variance, and intensity range variance
    """

    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape) # Placeholder of the filtered image
    
    # Todo: For each pixel position [i, j], you need to compute the filtered output: img_filtered[i, j] using a non-local means filter
    # step 1: compute window_sizexwindow_size filter weights of the non-local means filter in terms of intensity_variance. 
    # step 2: compute the filtered pixel img_filtered[i, j] using the obtained kernel weights and the pixel values in the search window
    # Please see slides 30 and 31 of lecture 6. Clarification: the patch_size refers to the size of small image patches (image content in yellow, 
    # red, and blue boxes in the slide 30); intensity_variance denotes sigma^2 in slide 30; the window_size is the size of the search window as illustrated in slide 31.
    # Tip: use zero-padding to address the black border issue. 

    # ********************************
    # Your code is here.
    # ********************************
    # Inside the nlm_filtering function



    for i in range(window_size // 2, img.shape[0] - window_size // 2):
        for j in range(window_size // 2, img.shape[1] - window_size // 2):
            # Extract the central patch at (i, j)
            patch = img[i - patch_size // 2:i + patch_size // 2 + 1, j - patch_size // 2:j + patch_size // 2 + 1]

            # Step 1: Compute the weights for each pixel in the window based on intensity similarity
            weights = np.zeros((window_size, window_size), dtype=np.float32)

            for u in range(window_size):
                for v in range(window_size):
                    # Extract the window patch centered at (i + u - window_size // 2, j + v - window_size // 2)
                    window_patch = img[i + u - window_size // 2 - patch_size // 2:i + u - window_size // 2 + patch_size // 2 + 1,
                                      j + v - window_size // 2 - patch_size // 2:j + v - window_size // 2 + patch_size // 2 + 1]

                    if window_patch.shape != patch.shape:
                        # Handle the case where the window_patch size doesn't match the patch size
                        continue

                    # Compute the squared difference between the central patch and the window patch
                    squared_diff = np.sum(np.square(patch - window_patch))

                    # Compute the weight using the intensity variance parameter
                    weight = np.exp(-squared_diff / (2.0 * intensity_variance ** 2))

                    # Assign the weight to the corresponding position in the weights matrix
                    weights[u, v] = weight

            # Step 2: Compute the filtered pixel value img_filtered[i, j] using the weights and pixel values in the window
            weighted_sum = 0.0
            weight_sum = 0.0

            for u in range(window_size):
                for v in range(window_size):
                    # Extract the pixel value at (i + u - window_size // 2, j + v - window_size // 2)
                    pixel_value = img[i + u - window_size // 2, j + v - window_size // 2]

                    # Retrieve the weight for this pixel from the 'weights' matrix
                    weight = weights[u, v]

                    # Accumulate the weighted sum of pixel values
                    weighted_sum += weight * pixel_value

                    # Accumulate the sum of weights
                    weight_sum += weight

            # Compute the filtered pixel value by dividing the weighted sum by the sum of weights
            img_filtered[i, j] = weighted_sum / weight_sum

    img_filtered = img_filtered * 255
    img_filtered = np.uint8(img_filtered)
    return img_filtered

 
if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA) # reduce image size for saving your computation time
    cv2.imwrite('results/im_original.png', img) # save image 
    
    # Generate Gaussian noise
    noise = np.random.normal(0,0.6,img.size)
    noise = noise.reshape(img.shape[0],img.shape[1]).astype('uint8')
   
    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)
    
    # Bilateral filtering
    intensity_variance = 1
    patch_size = 5 # small image patch size
    window_size = 15 # serach window size
    img_bi = nlm_filtering(img_noise, intensity_variance, patch_size, window_size)
    cv2.imwrite('results/im_nlm456.png', img_bi)

In [14]:
"""
CS 4391 Homework 2 Programming: Part 4 - non-local means filter
Implement the nlm_filtering() function in this python script
"""
 
import cv2
import numpy as np
import math

def nlm_filtering(
    img: np.uint8,
    intensity_variance: float,
    patch_size: int,
    window_size: int,
) -> np.uint8:
    """
    Homework 2 Part 4
    Compute the filtered image given an input image, kernel size of image patch, spatial variance, and intensity range variance
    """

    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape) # Placeholder of the filtered image
    
    # Todo: For each pixel position [i, j], you need to compute the filtered output: img_filtered[i, j] using a non-local means filter
    # step 1: compute window_sizexwindow_size filter weights of the non-local means filter in terms of intensity_variance. 
    # step 2: compute the filtered pixel img_filtered[i, j] using the obtained kernel weights and the pixel values in the search window
    # Please see slides 30 and 31 of lecture 6. Clarification: the patch_size refers to the size of small image patches (image content in yellow, 
    # red, and blue boxes in the slide 30); intensity_variance denotes sigma^2 in slide 30; the window_size is the size of the search window as illustrated in slide 31.
    # Tip: use zero-padding to address the black border issue. 

    # ********************************
    # Your code is here.
    # ********************************
    # Inside the nlm_filtering function


    padding = window_size // 2

    # Apply zero-padding to the input image
    img_padded = np.pad(img, ((padding, padding), (padding, padding)), mode='constant', constant_values=0.0)
    for i in range(window_size // 2, img.shape[0] - window_size // 2):
        for j in range(window_size // 2, img.shape[1] - window_size // 2):
            # Extract the central patch at (i, j)
            patch = img[i - patch_size // 2:i + patch_size // 2 + 1, j - patch_size // 2:j + patch_size // 2 + 1]

            # Step 1: Compute the weights for each pixel in the window based on intensity similarity
            weights = np.zeros((window_size, window_size), dtype=np.float32)

            for u in range(window_size):
                for v in range(window_size):
                    # Extract the window patch centered at (i + u - window_size // 2, j + v - window_size // 2)
                    window_patch = img[i + u - window_size // 2 - patch_size // 2:i + u - window_size // 2 + patch_size // 2 + 1,
                                      j + v - window_size // 2 - patch_size // 2:j + v - window_size // 2 + patch_size // 2 + 1]

                    if window_patch.shape != patch.shape:
                        # Handle the case where the window_patch size doesn't match the patch size
                        continue

                    # Compute the squared difference between the central patch and the window patch
                    squared_diff = np.sum(np.square(patch - window_patch))

                    # Compute the weight using the intensity variance parameter
                    weight = np.exp(-squared_diff / (2.0 * intensity_variance ** 2))

                    # Assign the weight to the corresponding position in the weights matrix
                    weights[u, v] = weight

            # Step 2: Compute the filtered pixel value img_filtered[i, j] using the weights and pixel values in the window
            weighted_sum = 0.0
            weight_sum = 0.0

            for u in range(window_size):
                for v in range(window_size):
                    # Extract the pixel value at (i + u - window_size // 2, j + v - window_size // 2)
                    pixel_value = img[i + u - window_size // 2, j + v - window_size // 2]

                    # Retrieve the weight for this pixel from the 'weights' matrix
                    weight = weights[u, v]

                    # Accumulate the weighted sum of pixel values
                    weighted_sum += weight * pixel_value

                    # Accumulate the sum of weights
                    weight_sum += weight

            # Compute the filtered pixel value by dividing the weighted sum by the sum of weights
            img_filtered[i, j] = weighted_sum / weight_sum
    
    img_filtered = img_filtered[padding:-padding, padding:-padding]
    
    img_filtered = img_filtered * 255
    img_filtered = np.uint8(img_filtered)
    return img_filtered

 
if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA) # reduce image size for saving your computation time
    cv2.imwrite('results/im_original.png', img) # save image 
    
    # Generate Gaussian noise
    noise = np.random.normal(0,0.6,img.size)
    noise = noise.reshape(img.shape[0],img.shape[1]).astype('uint8')
   
    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)
    
    # Bilateral filtering
    intensity_variance = 1
    patch_size = 5 # small image patch size
    window_size = 15 # serach window size
    img_bi = nlm_filtering(img_noise, intensity_variance, patch_size, window_size)
    cv2.imwrite('results/im_nlmtired.png', img_bi)

In [ ]:
import cv2
import numpy as np
import math
import sys
 

def nlm_filtering(
    img: np.uint8,
    intensity_variance: float,
    patch_size: int,
    window_size: int,
) -> np.uint8:
    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape) # Placeholder of the filtered image

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            patch = img[max(0, i - patch_size):min(img.shape[0], i + patch_size + 1),
            max(0, j - patch_size):min(img.shape[1], j + patch_size + 1)]

            weighted_sum = 0
            normalization_factor = 0

            for m in range(max(0, i - window_size), min(img.shape[0], i + window_size + 1)):
                for n in range(max(0, j - window_size), min(img.shape[1], j + window_size + 1)):
                    neighbor_patch = img[max(0, m - patch_size):min(img.shape[0], m + patch_size + 1),
                                         max(0, n - patch_size):min(img.shape[1], n + patch_size + 1)]

                    if patch.shape == neighbor_patch.shape:
                        diff = patch - neighbor_patch
                        diff_squared = np.sum(diff ** 2)

                        weight = math.exp(-diff_squared / (2 * intensity_variance ** 2))
                        weighted_sum += weight * img[m, n]
                        normalization_factor += weight

            img_filtered[i, j] = weighted_sum / normalization_factor

    img_filtered = np.uint8(img_filtered * 255)
    return img_filtered

if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA) # reduce image size for saving your computation time
    cv2.imwrite('results/im_original.png', img) # save image 
    
    # Generate Gaussian noise
    noise = np.random.normal(0,0.6,img.size)
    noise = noise.reshape(img.shape[0],img.shape[1]).astype('uint8')
   
    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)
    
    # NLM filtering
    intensity_variance = 1
    patch_size = 5 # small image patch size
    window_size = 15 # search window size
    img_nlm = nlm_filtering(img_noise, intensity_variance, patch_size, window_size)
    cv2.imwrite('results/im_nlm2.png', img_nlm)


In [ ]:
"""
CS 4391 Homework 2 Programming: Part 4 - non-local means filter
Implement the nlm_filtering() function in this python script
"""
 
import cv2
import numpy as np
import math

def nlm_filtering(
    img: np.uint8,
    intensity_variance: float,
    patch_size: int,
    window_size: int,
) -> np.uint8:
    
    """
    Homework 2 Part 4
    Compute the filtered image given an input image, kernel size of image patch, spatial variance, and intensity range variance
    """

    img = img / 255
    img = img.astype("float32")
    img_filtered = np.zeros(img.shape) # Placeholder of the filtered image
    
    # Todo: For each pixel position [i, j], you need to compute the filtered output: img_filtered[i, j] using a non-local means filter
    # step 1: compute window_sizexwindow_size filter weights of the non-local means filter in terms of intensity_variance. 
    # step 2: compute the filtered pixel img_filtered[i, j] using the obtained kernel weights and the pixel values in the search window
    # Please see slides 30 and 31 of lecture 6. Clarification: the patch_size refers to the size of small image patches (image content in yellow, 
    # red, and blue boxes in the slide 30); intensity_variance denotes sigma^2 in slide 30; the window_size is the size of the search window as illustrated in slide 31.
    # Tip: use zero-padding to address the black border issue. 
    # ********************************
    # Your code is here.
    # ********************************
    

    pad = np.pad(img, (((window_size // 2), (window_size // 2)), ((window_size // 2), (window_size // 2))), mode='constant')
    h1, w1 = img.shape
    for i in range((window_size // 2), h1 - (window_size // 2)):
        for j in range(window_size // 2, w1 - window_size // 2):
            patch = img[i - (patch_size // 2):i + (patch_size // 2) + 1, 
                        j - (patch_size // 2):j + (patch_size // 2) + 1]
            filter_weight = np.zeros((window_size, window_size), dtype=np.float32)

            for u in range(window_size):
                for v in range(window_size):
                    window_patch = img[i + u - (window_size // 2) - (patch_size // 2):i + u - (window_size // 2) + (patch_size // 2) + 1, 
                                       j + v - (window_size // 2) - (patch_size // 2):j + v - (window_size // 2) + (patch_size // 2)+ 1]
                    if window_patch.shape == patch.shape:
                        filter_weight[u, v] = (np.exp(-(np.sum(np.square(patch - window_patch))) / (2.0 * intensity_variance ** 2)))

            sum3 = 0
            kernel_weight1 = 0
            for u in range(window_size):
                for v in range(window_size):
                    sum3 += filter_weight[u, v] * img[i + u - (window_size // 2), j + v - (window_size // 2)]
                    kernel_weight1 += filter_weight[u, v]

            img_filtered[i, j] = sum3 / kernel_weight1
    
    img_filtered = img_filtered[(window_size // 2):-(window_size // 2), (window_size // 2):-(window_size // 2)]
    img_filtered = img_filtered * 255
    img_filtered = np.uint8(img_filtered)
    return img_filtered

 
if __name__ == "__main__":
    img = cv2.imread("data/img/butterfly.jpeg", 0) # read gray image
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA) # reduce image size for saving your computation time
    cv2.imwrite('results/im_original.png', img) # save image 
    
    # Generate Gaussian noise
    noise = np.random.normal(0,0.6,img.size)
    noise = noise.reshape(img.shape[0],img.shape[1]).astype('uint8')
   
    # Add the generated Gaussian noise to the image
    img_noise = cv2.add(img, noise)
    cv2.imwrite('results/im_noisy.png', img_noise)
    
    # Bilateral filtering
    intensity_variance = 1
    patch_size = 5 # small image patch size
    window_size = 15 # serach window size
    img_bi = nlm_filtering(img_noise, intensity_variance, patch_size, window_size)
    cv2.imwrite('results/im_nlmgr.png', img_bi)